In [1]:
# %%
import sys
sys.path.append('/Volumes/GoogleDrive/My Drive/GemPhy/GP_old')

import numpy as np
import gempy as gp
from gempy.core.tensor.modeltf_var import ModelTF
from gempy.assets.geophysics import *
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

PYVISTA_IMPORT True


In [4]:
# %%
path_to_data = "/Users/zhouji/Documents/github/PatuaModel/Data/unconformity/"

geo_data = gp.create_data( extent=[0, 1000, 0, 1000, 0, 1000], resolution=[50, 50, 50],
                          path_o=path_to_data + "model_intrusion_reduce_orientations.csv",
                          path_i=path_to_data + "model_intrusion_reduce_surface_points.csv")

# %%
gp.map_series_to_surfaces(geo_data, {"Strat_Series1": ('rock3'),
                                    "Strat_Series2": ('rock2', 'rock1'),
                                    "Basement_Series": ('basement')})


mapping_object = {'Strat_Series1':   np.array([1,1,0.01]),
                'Strat_Series2':   np.array([1,1,1]),
                'Basement_Series': np.array([1,1,1])}
gp.assign_global_anisotropy(geo_data,mapping_object)

# %%
model = ModelTF(geo_data)
model.activate_regular_grid()
model.create_tensorflow_graph(gradient = False)

# %%
model.compute_model()

# %%
gp._plot.plot_3d(model)

Active grids: ['regular']


/Volumes/GoogleDrive/My Drive/GemPhy/GP_old/gempy/core/tensor/modeltf_var.py:572: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  [final_block[l0: l1].numpy(), Z_x[:, l0: l1].numpy().astype(float)])


In [6]:
model.anisotropy_vector

<tf.Tensor: shape=(3, 3), dtype=float64, numpy=
array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])>

In [11]:
np.stack(model.geo_data.series.df['anisotropy'].values, axis = 0)

array([[1. , 1. , 0.1],
       [1. , 1. , 1. ],
       [1. , 1. , 1. ]])

In [5]:

mapping_object = {'Strat_Series1':   np.array([1,1,0.1]),
                'Strat_Series2':   np.array([1,1,1]),
                'Basement_Series': np.array([1,1,1])}
geo_data.assign_global_anisotropy(mapping_object)

geo_data.series.df

,order_series,BottomRelation,isActive,anisotropy
Strat_Series1,1,Erosion,True,"[1.0, 1.0, 0.1]"
Strat_Series2,2,Erosion,True,"[1, 1, 1]"
Basement_Series,3,Erosion,False,"[1, 1, 1]"


In [120]:

mapping_object = {'Strat_Series1':   [1,1,0.1],
                'Strat_Series2':   [1,1,1],
                'Basement_Series': [1,1,1]}
# anisotropy_df = pd.DataFrame(index=mapping_object.keys(), columns=['anisotropy'])
# for index in anisotropy_df.index:
#     anisotropy_df.at[index, 'anisotropy'] = mapping_object[index]

# pd.merge(geo_data.series.df, anisotropy_df, left_index=True, right_index=True)

anisotropy_df = pd.DataFrame(index=mapping_object.keys(), columns=['anisotropy'])
for index in anisotropy_df.index:
    anisotropy_df.at[index, 'anisotropy'] = mapping_object[index]

geo_data.series.df = pd.merge(geo_data.series.df, anisotropy_df, left_index=True, right_index=True)


In [125]:
anisotropy_df = pd.DataFrame(index=mapping_object.keys(), columns=['anisotropy'])
for index in anisotropy_df.index:
    anisotropy_df.at[index, 'anisotropy'] = mapping_object[index]

pd.merge(geo_data.series.df, anisotropy_df, left_index=True, right_index=True)

,order_series,BottomRelation,isActive,anisotropy
Strat_Series1,1,Erosion,True,"[1, 1, 0.1]"
Strat_Series2,2,Erosion,True,"[1, 1, 1]"
Basement_Series,3,Erosion,False,"[1, 1, 1]"


In [98]:
geo_data.series.df['anisotropy']

Strat_Series1      None
Strat_Series2      None
Basement_Series    None
Name: anisotropy, dtype: object